**The code below scrapes data from LottoMaxNumbers.com and builds a pandas dataframe for the prize breakdown from 2012 to 2023.**

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
import re

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/A Python Learning/Lotto649/649.xlsx')

In [ ]:
df.head()

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,1982-06-12,3,11,12,14,41,43,13
1,649,2,0,1982-06-19,8,33,36,37,39,41,9
2,649,3,0,1982-06-26,1,6,23,24,27,39,34
3,649,4,0,1982-07-03,3,9,10,13,20,43,34
4,649,5,0,1982-07-10,5,14,21,31,34,47,45


In [ ]:
df['Date'] = df['DRAW DATE'].dt.strftime('%Y-%m-%d')
filtered_dfafter2008 = df[df['DRAW DATE'].dt.year >= 2012]
filtered_dfafter2008dec = filtered_dfafter2008[filtered_dfafter2008['DRAW DATE'] < pd.to_datetime('2025-01-01')]

# Create a new 'Date' column with the formatted date for filtered_dfafter2008dec
filtered_dfafter2008dec['Date'] = filtered_dfafter2008dec['DRAW DATE'].dt.strftime('%Y-%m-%d')

In [ ]:
trial = filtered_dfafter2008dec['Date'].tolist()
print(trial)

['2012-01-04', '2012-01-07', '2012-01-11', '2012-01-14', '2012-01-18', '2012-01-21', '2012-01-25', '2012-01-28', '2012-02-01', '2012-02-04', '2012-02-08', '2012-02-11', '2012-02-15', '2012-02-18', '2012-02-22', '2012-02-25', '2012-02-29', '2012-03-03', '2012-03-07', '2012-03-10', '2012-03-14', '2012-03-17', '2012-03-21', '2012-03-24', '2012-03-28', '2012-03-31', '2012-04-04', '2012-04-07', '2012-04-11', '2012-04-14', '2012-04-18', '2012-04-21', '2012-04-25', '2012-04-28', '2012-05-02', '2012-05-05', '2012-05-09', '2012-05-12', '2012-05-16', '2012-05-19', '2012-05-23', '2012-05-26', '2012-05-30', '2012-06-02', '2012-06-06', '2012-06-09', '2012-06-13', '2012-06-16', '2012-06-20', '2012-06-23', '2012-06-27', '2012-06-30', '2012-07-04', '2012-07-07', '2012-07-11', '2012-07-14', '2012-07-18', '2012-07-21', '2012-07-25', '2012-07-28', '2012-08-01', '2012-08-04', '2012-08-08', '2012-08-11', '2012-08-15', '2012-08-18', '2012-08-22', '2012-08-25', '2012-08-29', '2012-09-01', '2012-09-05', '2012

In [ ]:
column_titles = ['Date','Draw',	'Winners',	'Prize',	'Breakdown']
Structure = []
column1 = []
column2 = []
column3 = []
column4 =[]
p =[]

for date in trial:
  url = 'https://www.lottonumbers.com/canada-6-49/results/'+date
  html_data = requests.get(url).text
  soup = BeautifulSoup(html_data, 'html.parser')
  tables = soup.find_all('table',{'class':'table-breakdown'})
  for table in tables:
    row_data = table.find_all('td')
    roww = [data.text.strip() for data in row_data]
    a = roww[1]
    for i in range(0, len(roww), 4):
      column1.append(roww[i])
      column2.append(roww[i + 1])
      column3.append(roww[i + 2])
      column4.append(roww[i + 3])
    p.append(date)
    data = {'Draw': column1, 'Prize': column2, 'Winners': column3}
    time.sleep(0.2)
df = pd.DataFrame(data)
df
dates = [date for date in trial for _ in range(9)]

gg = pd.DataFrame()
gg['Date'] = dates
bb = pd.merge(gg,df,left_index=True,right_index=True)
bb

,Date,Draw,Prize,Winners
0,2012-01-04,Match 6,"CAD $3,052,345.00",Rollover\r\n\t\t\t\t\t\t\t\t\t\t\t\t0
1,2012-01-04,Match 5 plus Bonus,"CAD $72,674.90",3
2,2012-01-04,Match 5,"CAD $1,765.80",102
3,2012-01-04,Match 4,CAD $50.30,"6,787"
4,2012-01-04,Match 3,CAD $10.00,"110,913"
...,...,...,...,...
11335,2024-01-27,Match 3,CAD $10.00,"102,663"
11336,2024-01-27,Match 2 plus Bonus,CAD $5.00,"74,656"
11337,2024-01-27,Match 2,CAD $0.00,"656,172"
11338,2024-01-27,Gold Ball Jackpot,"CAD $1,000,000.00",Rollover\r\n\t\t\t\t\t\t\t\t\t\t\t\t1


In [ ]:
a = bb

In [ ]:
interval = 9
rows_to_drop = range(interval - 1, len(bb), interval)
bb = bb.drop(rows_to_drop).reset_index(drop=True)

In [ ]:
bb['Winners'] = bb['Winners'].astype(str)
bb['Winners'] = bb['Winners'].str.replace('[^\d]', '', regex=True)
bb['Winners'] = bb['Winners'].replace('', 0)
bb['Winners'] = bb['Winners'].astype(int)

In [ ]:
bb.head(10)

,Date,Draw,Prize,Winners
0,2012-01-04,Match 6,"CAD $3,052,345.00",0
1,2012-01-04,Match 5 plus Bonus,"CAD $72,674.90",3
2,2012-01-04,Match 5,"CAD $1,765.80",102
3,2012-01-04,Match 4,CAD $50.30,6787
4,2012-01-04,Match 3,CAD $10.00,110913
5,2012-01-04,Match 2 plus Bonus,CAD $5.00,60552
6,2012-01-04,Match 2,CAD -,0
7,2012-01-04,Guaranteed Prize Draw,CAD -,0
8,2012-01-07,Match 6,"CAD $6,732,159.00",0
9,2012-01-07,Match 5 plus Bonus,"CAD $52,568.80",5


In [ ]:
bb['Prize'] = bb['Prize'].astype(str)
bb['Prize'] = bb['Prize'].str.replace('[^\d\.]', '', regex=True)
bb['Prize'] = bb['Prize'].replace('', 0)
bb['Prize'] = bb['Prize'].astype(float).astype(int)

In [ ]:
bb.tail(20)

,Date,Draw,Prize,Winners
10060,2024-01-20,Match 3,10,95177
10061,2024-01-20,Match 2 plus Bonus,5,58570
10062,2024-01-20,Match 2,0,630911
10063,2024-01-20,Gold Ball Jackpot,1000000,1
10064,2024-01-24,Match 6,5000000,0
10065,2024-01-24,Match 5 plus Bonus,69978,3
10066,2024-01-24,Match 5,1546,57
10067,2024-01-24,Match 4,78,4510
10068,2024-01-24,Match 3,10,83150
10069,2024-01-24,Match 2 plus Bonus,5,50116


In [ ]:
bb['Winners'] = bb['Winners'].astype(str)
bb['Winners'] = bb['Winners'].str.replace('[^\d]', '', regex=True)
bb['Winners'] = bb['Winners'].astype(int)
bb

bb['Prize'] = bb['Prize'].astype(str)
bb['Prize'] = bb['Prize'].str.replace('[^\d\.]', '', regex=True)
interval = 9
rows_to_drop = range(interval - 1, len(bb), interval)
bb = bb.drop(rows_to_drop).reset_index(drop=True)
bb['Prize'] = bb['Prize'].astype(float).astype(int)

bb.to_csv('/content/drive/MyDrive/A Python Learning/Lotto649_2008to2024onwards.csv',index=False)
bb

ValueError: invalid literal for int() with base 10: ''

In [ ]:
bb.head(10)

In [ ]:
for table in tables[4:5]:
    row_data = table.find_all('td')
    roww = [data.text.strip() for data in row_data]
  for i in range(0, len(roww), 3):
    column1.append(roww[i])
    column2.append(roww[i + 1])
    column3.append(roww[i + 2])
  p.append(date)
  data = {'Draw': column1, 'Winners': column2, 'Prize': column3}
  time.sleep(0.2)
df = pd.DataFrame(data)

# Repeat each date eight times
dates = [date for date in trial for _ in range(7)]

# Create a DataFrame with the 'Date' column
gg = pd.DataFrame()
gg['Date'] = dates
bb = pd.merge(gg,df,left_index=True,right_index=True)

bb['Winners'] = bb['Winners'].astype(str)
bb['Winners'] = bb['Winners'].str.replace('[^\d]', '', regex=True)
bb['Winners'] = bb['Winners'].astype(int)

In [ ]:
bb.head(10)

In [ ]:
for i in range(1, len(bb), 7):
  a = str(bb.at[i, 'Winners'])
  if len(str(bb.at[i,'Winners'])) == 2:
    bb.at[i, 'Winners'] = int(a[0]) + int(a[1])
  elif len(str(bb.at[i,'Winners'])) == 3:
    bb.at[i, 'Winners'] = int(a[0]) + int(a[1]) + int(a[2])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
bb.to_csv('/content/drive/MyDrive/A Python Learning/Lotto649_2012to2024.csv',index=False)

In [ ]:
bb.head()